# Интерфейсы scikit-learn

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import numpy as np
from numpy.typing import NDArray

## Estimator

Для примера построим простой estimator, который в перспективе будет вычитать из признаков их среднее значение и после сдвигать признаки на заранее заданную константу

In [ ]:
class SubtractMeanAndShiftEstimator(BaseEstimator):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

In [ ]:
m = SubtractMeanAndShiftEstimator(shift=3)

Метод `get_params()` реализован в `BaseEstimator`, и мы можем сразу использовать его для получения гиперпараметров модели. Это возможно, так как единственный гиперпараметр `shift` был передан как явное ключевое слово в контрукторе

Обратите внимание, что соответствующий аттрибут класса должен совпадать с ключевым словом: `self.shift = shift`

In [ ]:
m.get_params()

Аналогично мы можем использовать `set_params()` для задания значений гиперпараметров. Этот метод пригодится при поиске оптимальных значений гиперпараметров

In [ ]:
m.set_params(shift=5)
m.get_params()

In [ ]:
X = np.array([
    [1, 10],
    [3, 30],
    [2, 20],
])
y = np.array([
    [ 0, -8],
    [ 2, 10],
    [ 1,  1],
])
m.fit(X, y)

## Predictor

Рассмотрим тот же класс, но добавим к нему методы `predict()` и `score()`

In [ ]:
class SubtractMeanAndShiftPredictor(BaseEstimator):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def predict(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X -  e @ self.means_.reshape(-1, 1).T + self.shift

    def score(self, X: NDArray, y: NDArray) -> float:
        return r2_score(y, self.predict(X))  # R2 \in (-\infty; 1] is the coefficient of determination

Так как мы специально добавили небольшое отклонение в y, наш R2 чуть меньше 1

In [ ]:
model = SubtractMeanAndShiftPredictor(shift=1)
model.fit(X)
model.predict(X)
model.score(X, y)

## Transformer

Рассмотрим тот же класс, но добавим к нему метод `transform()`

In [ ]:
class SubtractMeanAndShiftTransformer(BaseEstimator, OneToOneFeatureMixin, TransformerMixin):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def transform(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X -  e @ self.means_.reshape(-1, 1).T + self.shift

In [ ]:
t = SubtractMeanAndShiftTransformer(shift=5)
t.fit(X)
t.transform(X)

Так как мы добавили `TransformerMixin`, мы можем использовать метод `fit_transform()`, не реализуя его явно

In [ ]:
t.fit_transform(X)

Аналогично мы можем использовать метод `get_feature_names_out()`, так как мы добавили `OneToOneFeatureMixin`

In [ ]:
t.get_feature_names_out(input_features=['x', 'y'])

## Pipelines

С помощью Pipeline мы можем производить последовательную обработку данных и выполнять предсказание в конце

In [ ]:
X = np.array([
    [1, 10],
    [3, 30],
    [2, 20],
])
y = np.array([
    [0],
    [2],
    [1],
])

pipeline = Pipeline([
    ("shifter", SubtractMeanAndShiftTransformer(shift=5)),
    ("regressor", LinearRegression()),
])
...
pipeline.fit(X, y)
y_pred = pipeline.predict(X)
print(y_pred)

Pipeline хранит последовательные Estimators в аттрибуте `steps`

In [ ]:
pipeline.steps

Перейти к объекту i-го Estimator можно напрямую через `pipeline[i]`:

In [ ]:
pipeline[0]

In [ ]:
pipeline[1].coef_

Так как Pipeline сам является Estimator, мы можем увидеть список его параметров:

In [ ]:
pipeline.get_params()

Видно, параметры промежуточных Estimator указаны как `<estimator>__<parameter>`. Следовательно, мы можем изменить параметры любого промежуточного Estimator:

In [ ]:
pipeline.set_params(shifter__shift=10)
pipeline.get_params()